# Country Profiles Goals - Reports 

This notebook outlines how the two page reports we generate could be created using python. 

The process relies primarily on two different packages: 

1. Bokeh for visualization 
2. ReportLab for generate the pdf report

Both packages are free and available with a reasonably sized community to answer requests and questions. 

#### Import packages 

In [1]:
# libraries
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

## report lab
from reportlab.pdfgen import canvas
from reportlab.pdfgen.canvas import Canvas
from reportlab.lib.utils import ImageReader

from reportlab.pdfbase import pdfmetrics
from reportlab.pdfbase.ttfonts import TTFont

import sys
# import pyADVISE 
sys.path.append('/home/lightner/Documents/DevTech/Shared/')
from pyADVISE.report import inch, table, split, USAID_header
from pyADVISE import palettes

/home/lightner/miniconda3/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)


### Generate practice table

Here we generate practice tables for both the text and visuals.

#### Table DataFrames 

In [2]:
dictionary  = {#'country': ['Malawi']*7, 
              'variable': ['ID0']*7, 
              'row1': ['3,456']*4+['value1 value1']*3, 
              'year1':['year', 'year', 'year', 'year', '', '', ''], 
              'row2': ['value2']*4 +['']*3, 
              'year2': ['year2']*4+['']*3}

table_df = pd.DataFrame(dictionary) 

#define the order
table_df= table_df[['variable', 'row1', 'year1', 'row2', 'year2']]
# generate different sized tables based on table_df
df2 = table_df[['variable', 'row1']]
df3 = table_df.iloc[0:4,0:2]


In [3]:
table_df#; df2

,variable,row1,year1,row2,year2
0,ID0,"3,456",year,value2,year2
1,ID0,"3,456",year,value2,year2
2,ID0,"3,456",year,value2,year2
3,ID0,"3,456",year,value2,year2
4,ID0,value1 value1,,,
5,ID0,value1 value1,,,
6,ID0,value1 value1,,,


#### Visual Data

In [4]:
text = ['Health', 'Education', 'Gender', 'Macro', 'Finance', 'PSD', 'Conflict', 'Taxation', 'Democracy']
# example factor:
f1 = np.array([100, 140, 350, 400, 203, 504, 605, 304, 405])
f2 = np.array([60, 530, 540, 330, 200, 406, 709, 403, 509])
f3 = np.array([700, 400, 400, 600 , 320, 670, 302, 609, 304])


# data frame 
df = pd.DataFrame({'text': text, 'f1': f1, 'f2': f2, 'f3': f3})


def wide_to_long(df, cat_var, value_vars):
    ''' generate the dataset in ESDB long format.'''
    # reorder the dataframe is necessary for the rename code to work properly 
    # the wide to long function needs a common begining to each column to be shifted
    
    
    df = df[[cat_var] + value_vars]
    
    # rename columns 
    df.columns = [cat_var] + ['f'+str(f) for f in range(0, len(df.columns[1:]))]
    
    # move from wide to long format
    l = pd.wide_to_long(df, stubnames='f', i=[cat_var], j='var').reset_index()  
    
    return l

df_plot = wide_to_long(df, 'text', ['f1', 'f2', 'f3'])

df_plot.columns = ['series_name', 'country_id', 'value_start']
df_plot['year'] = '2017'
df_plot.head()

,series_name,country_id,value_start,year
0,Health,0,100,2017
1,Education,0,140,2017
2,Gender,0,350,2017
3,Macro,0,400,2017
4,Finance,0,203,2017


In [5]:
from pyADVISE.visualize import gen_grouped_bar
from bokeh.io import show
from bokeh.io import export_png


In [6]:
# selection of categories 
category_selection = ['Conflict', 'Democracy', 'Education', 'Finance', 'Gender',
       'Health', 'Macro', 'PSD', 'Taxation']
# selection country 
indicator_selection = [0,1,2]

### Import Font

In [10]:
font_file = '/home/lightner/fonts/Open_Sans/'

# generate font
files = ['OpenSans-Bold.ttf', 'OpenSans-BoldItalic.ttf', 'OpenSans-ExtraBoldItalic.ttf', 
         'OpenSans-ExtraBold.ttf', 'OpenSans-Italic.ttf', 'OpenSans-Light.ttf', 
         'OpenSans-Light.ttf', 'OpenSans-LightItalic.ttf', 'OpenSans-Regular.ttf',
          'OpenSans-SemiBold.ttf', 'OpenSans-SemiBoldItalic.ttf']

# generate names by take off the .ttf
names = [i[:-4] for i in files]


for i in range(0, len(files)): 
    pdfmetrics.registerFont(TTFont(names[i], font_file+files[i]))
    

### Color Scheme

In [11]:
palette = palettes.USAID_general()
palette

{'USAID Blue': '#002F6C',
 'USAID Red': '#BA0C2F',
 'Rich Black': '#212721',
 'Medium Blue': '#0067B9',
 'Light Blue': '#A7C6ED',
 'Dark Red': '#651D32',
 'Dark Gray': '#6C6463',
 'Medium Gray': '#8C8985',
 'Light Gray': '#CFCDC9',
 'White': 'white'}

### Functions to generate the tables 

In [26]:

def build_report(df, country='Malawi', date='July 2018'): 
    
    # create the canvas to place the pdf
    c = canvas.Canvas('./CP PDFs/'+country+".pdf")
    
    # set basics of an A4 for dimention reference and the margin choice 
    margin = 0.55
    height = 11.69
    width = 8.27
    
    # gap between columns 
    gap = 0.5
    column_w = (width-gap)/2
    
    # Number of characters for the line breaks in wrap for tables 
    split_num = 8
    
    #######################
    ######   Header 
    #######################
    
    c = USAID_header(c=c, country=country, date=date, height=height, margin=margin,
                    column_w=column_w, width=width, gap=gap)
    

    
    #################################
    #### generate tables 
    #################################
    
    #table 1 line_width, column_w,
    c, end_table1 = table(c=c, df=df, line_width=1.0, column_w=column_w, split_num=split_num,
                          second_line=True, height=height, margin=margin,
                        width=width, gap=gap, table_title = 'People',)
    
    
    # table 2
    start_table2 = (end_table1-inch(.8))/72
    c, end_table2 = table(c, df2, line_width=1.0, column_w=column_w, split_num=split_num,
                          second_line=False, height=height, margin=margin,
                        width=width, gap=gap, 
                          table_ystart = start_table2,
                       table_title = 'Government')
    
    
    #table 3
    start_table3 = (end_table2-inch(.8))/72
    c, end_table3 = table(c, df3,line_width=1.0, column_w=column_w, split_num=split_num,
                          second_line=False, height=height, margin=margin,
                        width=width, gap=gap ,
                       table_ystart = start_table3, table_title='Official Development Assistance')
      
    
    
    
    
    #################################
    ###  Add image 
    #################################
    
    title_plot='Title Bar Chart: '+country
    
    #### this is where we will build our bokeh images 
    # generatae the plot
    p = gen_grouped_bar(df_plot, cat_var_member=category_selection, 
               cat_var_group=indicator_selection, plot_dim=(1700, 4000), fill_alpha=1, 
                       title_text=title_plot)
    
    image1 = './Images/'+country+"_plot1.png"
    export_png(p, filename=image1)
    ### save them to a particular link 

    filename2 = './bokeh_plot(2).png'
    #c.drawImage(filename, 330, 0, width=250, height=250) 
    
    
    # then repeat
    
    c.drawImage(image1, 330, 160, width=250, height=106) 
    
    
    c.drawImage(filename2, 330, 410, width=250, height=250) 
    
    
    c.drawImage('./usaid3.png', inch(.05), 0, width=150, height=110) 
    
    
    ###################################
    #### Testing colors 
    ####################################
    '''
    # it is not sure whether the colors make the multiple transitions well, we see here 
    # grey box 
    c.setFillColor(palette['USAID Blue']) 
    c.setStrokeColor(palette['USAID Blue'])    
    c.rect(300, 150, 100, inch(.6), fill=1)
    '''
    ####################################
    #######   Footer
    ####################################
    
    c = USAID_footer_text(c)
    
    
    c.save()

In [27]:
build_report(table_df, country='Fire1')    

6.852000000000003
3.7140000000000026
1.7940000000000025


## Practice with loop

In [28]:
country_list = ['China', 'Mexico', 'Tanzania']

In [29]:
for i in country_list: 
    build_report(table_df, country=i)

6.852000000000003
3.7140000000000026
1.7940000000000025
6.852000000000003
3.7140000000000026
1.7940000000000025
6.852000000000003
3.7140000000000026
1.7940000000000025
